In [114]:
import datetime
import json
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

def getEventIndex(events, item):
  for i, ev in enumerate(events):
    if ev.get('description')!=None and item["description"] == ev["description"]:
      return i
  return -1

# Service Account Keyのロード
with open('credentials.json') as f:
    creds_data = json.load(f)

creds = Credentials.from_service_account_info(creds_data)
calendar_id = "do8vqrsgjkl70t0jvg3mbr9dh0@group.calendar.google.com"

# Google Calendar APIのserviceを作成
service = build('calendar', 'v3', credentials=creds)

event_result = (
    service.events()
    .list(
        calendarId=calendar_id,
        #                                     timeMin=st.isoformat()+'T00:00:00Z',
        #                                     timeMax=ed.isoformat()+'T00:00:00Z',
        singleEvents=True,
        orderBy="startTime",
    )
    .execute()
)
events = event_result.get("items", [])

def timeformat(t):            
    ret = t.get("dateTime", t.get("date"))
    return datetime.datetime.strptime(ret[:-6], "%Y-%m-%dT%H:%M:%S")
def timeformatNozone(t):            
    ret = t.get("dateTime", t.get("date"))
    return datetime.datetime.strptime(ret, "%Y-%m-%dT%H:%M:%S")

# Display all events
if not events:
    print("No upcoming events found.")
for i, event in enumerate(events):
    #start = event["start"].get("dateTime", event["start"].get("date"))
    #start = datetime.datetime.strptime(start[:-6], "%Y-%m-%dT%H:%M:%S")
    #start = event["start"]
    start = timeformat(event["start"])
    print("{:02}".format(i), start, event["summary"])
    if i > 10:
        break
print("Found items on Gcal",len(events))

00 2024-09-23 09:30:00 <Mango>導通/IT2朝会
01 2024-09-23 10:00:00 【経営研】業務(TSM)朝会
02 2024-09-23 11:00:00 <Mango>　導通バグ/IT2故障棚卸会
03 2024-09-23 15:00:00 <Mango>【業務G(TSM)】IT夕会
04 2024-09-23 16:00:00 FCS - 進捗報告
05 2024-09-23 18:00:00 <Mango>夕会：導通6(ST向け導通)に向けたリリース状況の確認
06 2024-09-24 09:30:00 <Mango>導通/IT2朝会
07 2024-09-24 10:30:00 DBAレビュークレンジング結果確認
08 2024-09-24 11:00:00 <Mango>　導通バグ/IT2故障棚卸会
09 2024-09-24 11:30:00 <Mango>【業務G(TSM)】管理定例
10 2024-09-24 12:15:00 ＜Mango＞業務G週次定例
11 2024-09-24 12:30:00 ＜Mango＞事前意識合わせ（JLIS個別会議に向けて）3/18～
count 208


In [116]:
def getEventIndex(events, item):
    for i, event in enumerate(events):
        if item["description"] == event["description"]:
            if item["summary"]!=event["summary"]:
                return i, False
            if timeformatNozone(item["start"])!=timeformat(event["start"]):
                return i, False
            if timeformatNozone(item["end"])!=timeformat(event["end"]):
                return i, False
            return i, True
    return -1, False
            
f = open('events.json','r')
items = json.load(f)
f.close()
print(len(items),"loaded")

for i, item in enumerate(items):
    index, same = getEventIndex(events,item)
    #print(i,(item["start"]),item["summary"])
    if index < 0:
        print(i,"insert",timeformatNozone(item["start"]),item["summary"])
        service.events().insert(
                calendarId=calendar_id,
                body=item).execute()
    else:
        event = events.pop(index)
        #if same:
        #    print(i,"skip",timeformatNozone(item["start"]),item["summary"])
        if not same:
            print(i,"update",timeformatNozone(item["start"]),item["summary"])
            service.events().update(
                    calendarId=calendar_id, 
                    eventId=event["id"], 
                    body=item
                ).execute()

for event in events:
    print("delete",timeformat(event["start"]),event["summary"])
    service.events().delete(
        calendarId=calendar_id, 
        eventId=event["id"]).execute()

print("done")

0 same 2024-09-23 09:30:00 <Mango>導通/IT2朝会
1 same 2024-09-23 10:00:00 【経営研】業務(TSM)朝会
2 same 2024-09-23 11:00:00 <Mango>　導通バグ/IT2故障棚卸会
3 same 2024-09-23 15:00:00 <Mango>【業務G(TSM)】IT夕会
4 same 2024-09-23 16:00:00 FCS - 進捗報告
5 same 2024-09-23 18:00:00 <Mango>夕会：導通6(ST向け導通)に向けたリリース状況の確認
6 same 2024-09-24 09:30:00 <Mango>導通/IT2朝会
7 same 2024-09-24 10:30:00 DBAレビュークレンジング結果確認
8 same 2024-09-24 11:00:00 <Mango>　導通バグ/IT2故障棚卸会
9 same 2024-09-24 11:30:00 <Mango>【業務G(TSM)】管理定例
10 same 2024-09-24 12:15:00 ＜Mango＞業務G週次定例
11 same 2024-09-24 12:30:00 ＜Mango＞事前意識合わせ（JLIS個別会議に向けて）3/18～
12 same 2024-09-24 13:00:00 【FCS】品質評価対応に関する振り返り – 中間報告+認識合わせMTG
13 same 2024-09-24 13:30:00 FCS - 進捗報告
14 same 2024-09-24 14:00:00 【J-LIS様/個別定例】SP-TSM（Acai対応）
15 same 2024-09-24 15:30:00 ※確定【MangoPJ】Acai対応 (J-LIS様個別定例)_振り返り
16 same 2024-09-24 16:00:00 ＜Mango＞社内進捗定例
17 same 2024-09-24 17:00:00 <Mango>【業務G(TSM)】開発定例
18 same 2024-09-24 17:30:00 <Mango>【業務G(TSM)】IT夕会
19 same 2024-09-24 18:00:00 【MangoPJ】定例資料Rv
20 same 2024-09-